In [67]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [4]:
# 경로 변수 (고정)

dataLoc = '../_data/'

trainfile = 'train_03_997.csv'
old_trainfile = 'train.csv'
kaggle_trainfile = 'train_k.csv'

testfile = 'test_clean.csv'
old_testfile = 'test.csv'

train_171016 = '171016_dow_hr_pas_L2_dur_only.csv'
train_osrm = '../_data/train_with_osrm.csv'
train_osrm_weather = '../_data/train_with_osrm_and_weather.csv'
train_osrm_weather_hourly = '../_data/train_with_osrm_and_weather_hourly.csv'
train_dropped_nan_fixed = '../_data/train__dropped__no_disaster_nan_fixed.csv'
train_added_airps = '../_data/train__dropped__lnglat_saved__airports__no_disaster.csv'
train_added_airps_cleaned = '../_data/train__dropped__cleaned__airports__no_disaster.csv'
train_added_airps_disas_cleaned = '../_data/train__dropped__cleaned__eval__airports__disaster.csv'
train_added_weekhr = 'train__dropped__cleaned__eval__a_d_weekhour.csv'
train_added_manh = '../_data/train__dropped__air_bliz_weekhr_manh.csv'
train_all_vars_rough = '../_data/train__all_vars__rough.csv'
train_latest = train_added_airps #701778 rows

In [5]:
pd.options.display.max_columns = 40

```
blizz
weekhour
```

In [6]:
# train_with_osrm_and_weather.csv
df_train = pd.read_csv(dataLoc + train_latest)
df_train.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,dur,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,False,430,Saturday,23,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,1454,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,1409,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,1081,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,800,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,1151,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


move duration to end

In [8]:
df_train = pd.concat([df_train.drop('dur', axis=1), df_train['dur']], axis=1)
df_train.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,430,Saturday,23,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,430,Saturday,23,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,430,Saturday,23,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,430,Saturday,23,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,430,Saturday,23,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [11]:
# init weekdays list

weekdays = list()
for i in range(7):
    time_ = pd.Timestamp(2017, 10, 16+i, 12, 0, 0)
    weekdays.append(time_.weekday_name)

weekdays

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [9]:
def get_weekhour(dow, hr):
    return weekdays.index(dow)*24 + hr

In [13]:
n = 10000
pd.concat([df_train[['dow', 'hr']],\
           pd.Series(np.vectorize(get_weekhour)(df_train['dow'], df_train['hr']), name='weekhour')],
          axis=1)[n:n+10]

,dow,hr,weekhour
10000,Friday,13,109
10001,Friday,13,109
10002,Friday,13,109
10003,Friday,13,109
10004,Friday,13,109
10005,Friday,13,109
10006,Friday,13,109
10007,Friday,13,109
10008,Friday,13,109
10009,Friday,13,109


In [16]:
df_copy = df_train.copy()

In [17]:
df_copy.insert(4, 'wkhr', pd.Series(np.vectorize(get_weekhour)(df_train['dow'], df_train['hr']), name='weekhour'))
df_copy.head()

,is_hol,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,430,Saturday,23,143,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,430,Saturday,23,143,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,430,Saturday,23,143,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,430,Saturday,23,143,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,430,Saturday,23,143,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


### todo: add `blizz` var
* 1/23 : 6
* 1/24 : 5
* 1/25 : 4
* 1/26 : 3
* 1/27 : 2
* 1/28 : 1
* others : 0

In [18]:
df_train['m_d'][df_train['m_d'] == 123]

577829    123
577830    123
577831    123
577832    123
577833    123
577834    123
577835    123
577836    123
577837    123
577838    123
577839    123
577840    123
577841    123
577842    123
577843    123
577844    123
577845    123
577846    123
577847    123
577848    123
577849    123
577850    123
577851    123
577852    123
577853    123
577854    123
577855    123
577856    123
577857    123
577858    123
         ... 
578994    123
578995    123
578996    123
578997    123
578998    123
578999    123
579000    123
579001    123
579002    123
579003    123
579004    123
579005    123
579006    123
579007    123
579008    123
579009    123
579010    123
579011    123
579012    123
579013    123
579014    123
579015    123
579016    123
579017    123
579018    123
579019    123
579020    123
579021    123
579022    123
579023    123
Name: m_d, Length: 1195, dtype: int64

In [21]:
def get_blizz(md):
    m = int(md[0])
    d = int(md[1:])
    blizz = 0
    if m == 1 and 23 <= d <= 28:
        blizz = 29 - d
    return blizz

df_copy.insert(1, 'blizz', df_train.loc[:, 'm_d'].apply(str).apply(get_blizz))

In [23]:
df_copy.loc[124 == df_train['m_d']].head()

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
575399,False,5,124,Sunday,23,167,0.069620,1,False,True,False,False,-73.863052,40.769279,-73.796631,40.790138,10307.5,756.3,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,1557
575400,False,5,124,Sunday,23,167,0.042868,1,False,False,False,False,-73.960403,40.761612,-73.921616,40.743359,4771.7,332.3,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,637
575401,False,5,124,Sunday,23,167,0.028081,5,False,False,False,False,-73.948738,40.781879,-73.965340,40.759232,2881.2,233.6,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,361
575402,False,5,124,Sunday,23,167,0.019552,1,False,False,False,False,-73.953217,40.775810,-73.967766,40.762749,2237.6,185.2,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,357
575403,False,5,124,Sunday,23,167,0.004694,1,False,False,False,False,-73.954590,40.765598,-73.954193,40.770275,689.4,60.7,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,221


In [25]:
df_copy.shape

(701778, 30)

In [26]:
df_copy.head()

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,0,430,Saturday,23,143,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,0,430,Saturday,23,143,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,0,430,Saturday,23,143,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,0,430,Saturday,23,143,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,0,430,Saturday,23,143,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [29]:
df_copy.columns

Index(['is_hol', 'blizz', 'm_d', 'dow', 'hr', 'wkhr', 'L2', 'pas_cnt', 'p_JFK',
       'p_LGA', 'd_JFK', 'd_LGA', 'p_lng', 'p_lat', 'd_lng', 'd_lat',
       'total_dist', 'total_time', 'day_max_temp', 'day_min_temp',
       'day_rainfall', 'day_snowfall', 'day_snowdepth', 'hr_temp', 'hr_humid',
       'hr_dewpt', 'hr_windspeed', 'hr_gustspeed', 'hr_precip', 'dur'],
      dtype='object')

In [31]:
df_copy.to_csv('../_data/train__701778__all_vars.csv', index=False)

In [33]:
df_copy.shape

(701778, 30)

In [40]:
from datetime import datetime
t0 = datetime.now()
mod = sm.OLS.from_formula('dur ~ C(is_hol)\
                               + C(dow):C(hr)\
                               + scale(total_dist) - 1', data=df_copy)
res = mod.fit()
print('done')
t1 = datetime.now()

print("{} sec".format((t1 - t0).seconds))

done
34 sec


In [41]:
res.df_resid

701608.0

In [42]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dur   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     28.73
Date:                Sun, 22 Oct 2017   Prob (F-statistic):               0.00
Time:                        20:53:38   Log-Likelihood:            -7.1835e+06
No. Observations:              701778   AIC:                         1.437e+07
Df Residuals:                  701608   BIC:                         1.437e+07
Df Model:                         169                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
C(is_hol)[False]                889.4907    103.873      8.563      0.000     685.903    1093.078
C(is_hol)[True]                 791.6034    116.100      6.818      0.000     564.051    1019.156
C(hr)[T.1]                      -99.4693    163.871     -0.607      0.544    -420.652     221.713
C(hr)[T.2]                     -119.0254    183.257     -0.649      0.516    -478.204     240.153
C(hr)[T.3]                       76.8590    202.404      0.380      0.704    -319.846     473.564
C(hr)[T.4]                      -47.9360    225.011     -0.213      0.831    -488.951     393.079
C(hr)[T.5]                     -310.8702    220.986     -1.407      0.160    -743.995     122.255
C(hr)[T.6]                     -279.4295    165.787     -1.685      0.092    -604.367      45.508
C(hr)[T.7]                      -15.0742    141.790     -0.106      0.915    -292.978     262.830
C(hr)[T.8]                      117.8850    137.801      0.855      0.392    -152.201     387.971
C(hr)[T.9]                      178.5555    139.535      1.280      0.201     -94.928     452.039
C(hr)[T.10]                     147.4085    143.368      1.028      0.304    -133.588     428.405
C(hr)[T.11]                     121.4345    142.469      0.852      0.394    -157.801     400.670
C(hr)[T.12]                     158.5252    140.395      1.129      0.259    -116.645     433.695
C(hr)[T.13]                     135.1894    140.327      0.963      0.335    -139.848     410.227
C(hr)[T.14]                     150.5207    137.760      1.093      0.275    -119.484     420.525
C(hr)[T.15]                     346.4467    139.761      2.479      0.013      72.520     620.374
C(hr)[T.16]                     191.3786    144.242      1.327      0.185     -91.331     474.088
C(hr)[T.17]                     132.4502    137.341      0.964      0.335    -136.735     401.635
C(hr)[T.18]                     127.9844    131.612      0.972      0.331    -129.971     385.940
C(hr)[T.19]                      77.4927    130.630      0.593      0.553    -178.537     333.523
C(hr)[T.20]                      40.5656    132.988      0.305      0.760    -220.086     301.218
C(hr)[T.21]                      11.9115    133.680      0.089      0.929    -250.097     273.920
C(hr)[T.22]                      31.0313    132.717      0.234      0.815    -229.089     291.152
C(hr)[T.23]                     129.6557    132.979      0.975      0.330    -130.978     390.290
C(dow)[T.Monday]:C(hr)[0]      -234.4637    182.817     -1.283      0.200    -592.780     123.853
C(dow)[T.Saturday]:C(hr)[0]       4.9852    135.614      0.037      0.971    -260.813     270.784
C(dow)[T.Sunday]:C(hr)[0]       -33.3833    136.526     -0.245      0.807    -300.970     234.204
C(dow)[T.Thursday]:C(hr)[0]    -181.2908    1

In [43]:
df_dropped = pd.DataFrame(columns=df_copy.columns)
df_dropped

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur


In [56]:
#dropping outliers by fox_critiria
def dropping_outliers1(fmla, data):
    model = sm.OLS.from_formula(fmla, data)
    result = model.fit()
    influence = result.get_influence()
    cooks_d2, pvals = influence.cooks_distance
    fox_cr = 4 / (result.df_resid -1)
    print(fox_cr)
    idx = np.where(cooks_d2 > fox_cr)[0]
    df_dropped = data.loc[idx].copy()
    data = data.drop(data.index[idx])
    print("Function dropping_outliers :" + str(len(idx)) + ' rows have beed deleted')
    return data, df_dropped

In [45]:
df_copy.shape

(701778, 30)

In [57]:
t0 = datetime.now()
fmla = 'dur ~ C(is_hol) + C(dow):C(hr) + scale(total_dist) - 1'

df_copy2, df_dropped = dropping_outliers1(fmla, df_copy)

t1 = datetime.now()

print(t1 - t0)

df_copy2.head()

5.70119739398e-06
Function dropping_outliers :972 rows have beed deleted
0:00:36.141879


,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,0,430,Saturday,23,143,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,0,430,Saturday,23,143,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,0,430,Saturday,23,143,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,0,430,Saturday,23,143,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,0,430,Saturday,23,143,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [58]:
df_dropped.head()

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
55,False,0,430,Saturday,23,143,0.020996,1,False,False,False,False,-74.005669,40.745209,-73.989098,40.758102,2512.4,309.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,86073
405,False,0,430,Saturday,22,142,0.010217,2,False,False,False,False,-73.976662,40.747765,-73.967712,40.752693,1265.4,135.2,65,46,0.0,0.0,0.0,11.1,0.53,1.7,5.6,0.0,0.0,86323
991,False,0,430,Saturday,21,141,0.014269,1,False,False,False,False,-73.991150,40.750629,-73.987946,40.736725,1889.9,265.7,65,46,0.0,0.0,0.0,11.7,0.47,0.6,11.1,0.0,0.0,86356
2617,False,0,430,Saturday,16,136,0.180187,1,True,False,False,False,-73.782318,40.644741,-73.944893,40.722439,20929.1,1333.6,65,46,0.0,0.0,0.0,16.7,0.34,0.6,11.1,0.0,0.0,26722
3201,False,0,430,Saturday,14,134,0.027076,5,False,False,False,False,-73.990387,40.737202,-74.012169,40.721119,3387.8,420.1,65,46,0.0,0.0,0.0,16.7,0.38,2.2,13.0,0.0,0.0,86317


In [69]:
top_5 = df_copy2['dur'].sort_values(ascending=False).head().index

In [70]:
df_copy2.loc[top_5]

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
342274,False,0,304,Friday,18,114,0.033882,1,False,False,False,False,-73.981911,40.778702,-73.957123,40.801800,4892.3,339.9,39,30,0.11,0.4,0.0,3.9,0.45,-7.2,18.5,31.5,0.0,18439
234110,False,0,322,Tuesday,19,43,0.126448,1,False,True,False,False,-73.870949,40.773830,-73.994087,40.745090,14809.7,1017.7,56,35,0.00,0.0,0.0,12.8,0.29,-5.0,13.0,0.0,0.0,15418
609652,False,0,117,Sunday,17,161,0.012918,1,False,False,False,False,-73.982101,40.755665,-73.991684,40.764328,2370.8,226.5,42,30,0.05,0.4,0.0,-0.6,0.76,-4.4,11.1,0.0,0.0,14243
78734,False,0,417,Sunday,11,155,0.010871,1,True,False,False,False,-73.790039,40.646923,-73.793320,40.657288,2416.6,266.7,75,44,0.00,0.0,0.0,13.3,0.37,-1.1,0.0,0.0,0.0,14160
533186,False,0,201,Monday,18,18,0.046676,1,False,False,False,False,-73.972336,40.761059,-73.998100,40.722137,5602.9,687.3,59,44,0.01,0.0,2.0,11.1,0.47,0.0,5.6,0.0,0.3,13850


In [62]:
df_dropped['dur'].sort_values(ascending=False)

460097    3526282
682215    2227612
460154    2049578
682482    1939736
608922      86369
316284      86366
126987      86364
498904      86364
203911      86362
355294      86362
49235       86360
47993       86358
411484      86358
485768      86357
377160      86357
991         86356
610941      86356
583168      86356
152055      86354
81787       86354
684086      86354
699197      86353
41224       86353
622087      86352
456425      86352
428877      86351
397182      86349
244366      86347
680950      86346
347460      86346
           ...   
37813       37305
511734      36949
600284      34920
427918      34916
552062      34014
568753      32338
356400      31751
684845      31099
336428      30291
640859      30062
669308      30018
195124      28575
669144      28241
517082      27217
91075       27064
433609      27047
2617        26722
66558       26577
462335      26396
324327      25713
568259      24351
368341      23440
8862        23096
109337      22012
538019    

In [48]:
df_copy2.shape

(700806, 30)

In [73]:
df_copy2.to_csv('../_data/train_700806_dropped_cooks.csv', index=False)

In [71]:
t0 = datetime.now()
modelz = sm.OLS.from_formula(fmla, df_copy2)
resz = modelz.fit()
print('done')
t1 = datetime.now()
print(t1-t0)

done
0:00:34.182396


```
adj rsq = 0.643
AIC ~ 1E7
Cond = 76.2
```

In [72]:
resz.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dur   R-squared:                       0.643
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     7462.
Date:                Sun, 22 Oct 2017   Prob (F-statistic):               0.00
Time:                        21:05:37   Log-Likelihood:            -5.1496e+06
No. Observations:              700806   AIC:                         1.030e+07
Df Residuals:                  700636   BIC:                         1.030e+07
Df Model:                         169                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
C(is_hol)[False]                699.1268      5.791    120.725      0.000     687.776     710.477
C(is_hol)[True]                 581.1905      6.472     89.797      0.000     568.505     593.876
C(hr)[T.1]                      -17.8175      9.133     -1.951      0.051     -35.719       0.084
C(hr)[T.2]                      -44.9929     10.214     -4.405      0.000     -65.012     -24.974
C(hr)[T.3]                      -55.5377     11.291     -4.919      0.000     -77.668     -33.408
C(hr)[T.4]                     -113.0417     12.550     -9.007      0.000    -137.640     -88.443
C(hr)[T.5]                     -220.5159     12.317    -17.903      0.000    -244.657    -196.375
C(hr)[T.6]                     -102.4339      9.237    -11.089      0.000    -120.539     -84.329
C(hr)[T.7]                       69.1176      7.903      8.746      0.000      53.628      84.607
C(hr)[T.8]                      201.3768      7.681     26.218      0.000     186.323     216.431
C(hr)[T.9]                      233.3335      7.778     30.000      0.000     218.089     248.578
C(hr)[T.10]                     232.4770      7.991     29.093      0.000     216.815     248.139
C(hr)[T.11]                     237.0367      7.940     29.852      0.000     221.474     252.600
C(hr)[T.12]                     221.2699      7.826     28.274      0.000     205.931     236.608
C(hr)[T.13]                     185.0435      7.822     23.656      0.000     169.712     200.375
C(hr)[T.14]                     238.5610      7.679     31.068      0.000     223.511     253.611
C(hr)[T.15]                     294.3145      7.793     37.767      0.000     279.041     309.588
C(hr)[T.16]                     271.5205      8.040     33.771      0.000     255.762     287.279
C(hr)[T.17]                     242.0371      7.655     31.619      0.000     227.034     257.040
C(hr)[T.18]                     202.6808      7.336     27.627      0.000     188.302     217.060
C(hr)[T.19]                     157.9105      7.281     21.687      0.000     143.639     172.182
C(hr)[T.20]                     114.2240      7.413     15.409      0.000      99.695     128.753
C(hr)[T.21]                      96.4639      7.451     12.946      0.000      81.860     111.068
C(hr)[T.22]                     121.1905      7.397     16.383      0.000     106.692     135.689
C(hr)[T.23]                     117.8986      7.414     15.902      0.000     103.368     132.430
C(dow)[T.Monday]:C(hr)[0]      -152.2822     10.189    -14.945      0.000    -172.253    -132.311
C(dow)[T.Saturday]:C(hr)[0]      72.0354      7.559      9.529      0.000      57.219      86.851
C(dow)[T.Sunday]:C(hr)[0]        70.0227      7.609      9.202      0.000      55.109      84.937
C(dow)[T.Thursday]:C(hr)[0]     -72.5029     